#### LIVRABLE PROJET PYTHON

#### INPUT
1) Import des libraires

In [ ]:
import os
from datetime import datetime 
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go

2) Création des dataframes à partir des fichiers csv

In [ ]:
def load_data_from_csv():

    notebook_dir = os.getcwd()                          #Chemin absolu du notebook
    data_dir = os.path.join(notebook_dir, "../data")    #Construction du chemin absolu vers le dossier 'data'
    csv_files = os.listdir(data_dir)                    #Liste des fichiers dans le dossier 'data'

    dataframes_dict = {}                                #Déclaration dictionnaire pour contenir les dataframes
    
    for file in csv_files:                              #Boucle pour chaque fichier .csv du dossier 'data' : 
        if file.endswith('.csv'):
            
            file_path = os.path.join(data_dir, file) 
            df = pd.read_csv(file_path)                         #Création du dataframe

            file_name = os.path.splitext(file)[0]
            dataframes_dict['df_' + str(file_name)] = df        #Renommage du dataframe avec préfixe "df_" + 'nom_du_fichier'
            
            df['timestamp'] = pd.to_datetime(df['timestamp'])   #Conversion 'timestamp' en type Datetime    

            print(f"df_{file_name}")                            #Affichage du df créé pour vérification
            print(df.head(3))   
            print("\n")

    return dataframes_dict

dataframes_dict = load_data_from_csv()

#### MANIPULATION DES DONNEES

1) Création d'un dataframe 'df_all' et intégration dans le dictionnaire

In [ ]:

def global_df_creation(): 
    
    dfs_to_concat = list(dataframes_dict.values())      #Identification des dataframes à concaténer
    print(f"Concaténation des dataframes :\n{list(dataframes_dict)}\n")
    
    df_all = pd.concat(dfs_to_concat, ignore_index=True)        #Concaténation verticale -> pas de perte de données avec la fonction concat, redéfinition des index pour avoir une clé unique par transaction
    print("'df_all' :")     
    display(df_all)     #Vérification : '2021-02-24 23:59:52.000' *2 dans le display = transactions conservées      

    dataframes_dict['df_all']= df_all       #Intégration de 'df_all' dans le dictionnaire
    print(f"Intégration dans le dictionnaire :\n{list(dataframes_dict)}\n")     
    
global_df_creation() 


#### CALCUL

1) Agrégation des données et calcul selon paramètres sélectionnés

In [ ]:
#_SELECTION_DES_PARAMETRES_#

def select_time_ladder(initial_frequency='60min'): #Widget select 'frequency'
    options = ['60min', '30min', '5min']                                                                        
    dropdown_ladder = widgets.Dropdown(options=options, description='Interval:', value=initial_frequency)       #Création du widget de sélection
    
    def on_change(change):   #Update
        if change['type'] == 'change' and change['name'] == 'value':        #En cas de changement de sélection, la nouvelle valeur correspond à la valeur sélectionnée
            clear_output()                                                  #Nettoyage pour MàJ affichage
            aggregation(dropdown_ladder.value, dropdown_vwmp_type.value)    #Aggrégation des données avec paramètres sélectionnés en tant qu'arguments
                
    dropdown_ladder.observe(on_change)      #Observation du changement de sélection
    return dropdown_ladder

def select_vwmp_type(initial_vwmp_type='lower'):    #Widget select 'vwmp_type'
    options = ['lower', 'upper']                                                                                    
    dropdown_vwmp_type = widgets.Dropdown(options=options, description='VWMP:', value=initial_vwmp_type)        #Création du widget de sélection
    
    def on_change(change):      #Update
       if change['type'] == 'change' and change['name'] == 'value':         #En cas de changement de sélection, la nouvelle valeur correspond à la valeur sélectionnée
            clear_output()                                                  #Nettoyage pour MàJ affichage
            aggregation(dropdown_ladder.value, dropdown_vwmp_type.value)    #Aggrégation des données avec paramètres sélectionnés en tant qu'arguments
            
    dropdown_vwmp_type.observe(on_change)       #Observation du changement de sélection
    return dropdown_vwmp_type

#_AGGREGATION_#

def aggregation(frequency, vwmp_type):      #Boucles de manipulations des dataframes (1 boucle calcul + 1 boucle cleaning)
    
    print("...calcul en cours")
    aggregated_dict = {}                        

    for key, df in dataframes_dict.items():     #Boucle de calcul (B1)
        aggregated_df = process_dataframe(key, df, frequency, vwmp_type) 
        aggregated_dict[key] = aggregated_df     

    for key, df in aggregated_dict.items():     #Boucle de cleaning (B2)
        cleaned_df = cleaning(df) 
        cleaned_dict[key] = cleaned_df 

    clear_output()
    visualisation(cleaned_dict)     #Fonction d'affichage des résultats sous forme de graphique en chandelier japonais + widgets de sélection

    return cleaned_dict
  
##__Boucle_1__## 

def process_dataframe(key, df, frequency, vwmp_type):   #Structure de calcul du dataframe
    
    exchange_name_splitted = (str(key)).split("df_")        #Extraction du nom de l'exchange
    exchange_name = exchange_name_splitted[1]

    df['weighted_volume'] = (df['price'] * df['amount'])        #Calcul du Weighted_Volume de chaque transaction avant agréggation

    aggregated_df = initial_aggregation(df, frequency)      #Agrégation des transactions initiales

    aggregated_df[f'{exchange_name}_vwap'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(calculate_vwap)                                           #Calcul du Volume Weighted Average Price [VWAP] 
    aggregated_df['ecart_type'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(calculate_ecart_type)                                                #Calcul de l'écart_type 
    aggregated_df[f'vwmp_{vwmp_type}'] = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).apply(lambda group: pd.Series(calculate_vwmp(group, vwmp_type)))    #Calcul du Volume Weighted Median Price [VWMP (lower or upper)] 

    return aggregated_df 

def initial_aggregation(df, frequency):     #Aggrégation initiale des transactions                          
            
    aggregated_df = df.groupby(pd.Grouper(key='timestamp', freq=frequency)).agg({    #Fonctions d'agrégation pour chaque colonnes
        'price': ['sum', 'first', 'max', 'min', 'last'],             
        'amount': 'sum',
        'weighted_volume': 'sum'
    })

    aggregated_df.columns = [       #Renommage des colonnes
                            'price', 'price_open', 'price_high', 'price_low', 'price_close',
                            'amount',
                            'weighted_volume'
    ]
    
    return aggregated_df

def calculate_vwap(group):      #Calcul du Volume Weighted Average Price [VWAP]   
    sum_price_amount = group['weighted_volume'].sum()                                           
    sum_amount = group['amount'].sum()

    if sum_amount != 0:
        return sum_price_amount / sum_amount 
    else : 
        return 0    

def calculate_ecart_type(group):        #Calcul de l'écart_type
    
    ecart_type = np.nanstd(group['price'])      #'np.nanstd' pour exclure les valeurs nulles dans le calcul de l'écart type 
    if ecart_type != np.nan :
        return ecart_type
    else : 
        return 0        #écart type à zéro dans le cas d'un échantillon de taille 1 (une seule valeur de 'price' sur la période)

def calculate_vwmp(group, vwmp_type):  #Calcul du Volume Weighted Median Price [VWMP (lower or upper)] 

    series_sorted = group.sort_values('amount')                         #Tri du volume par ordre croissant
    series_sorted['cumul_amount'] = series_sorted['amount'].cumsum()    #Calcul du volume cumulé
    total_volume_median = series_sorted['cumul_amount'].max() / 2       #Calcul de la médiane
    #--------
    if vwmp_type == 'lower':    #LOWER
       
        lower_cumulative_volume = series_sorted[series_sorted['cumul_amount'] <= total_volume_median]   #Filtrage de la série pour conserver la partie inférieure (>=) du volume cumulé 
       
        if not lower_cumulative_volume.empty:   #Si la taille de l'échantillon est >> 2
           
            max_cumul_amount_index = lower_cumulative_volume['cumul_amount'].idxmax()   #Identification de la ligne correspondante au volume cumulé maximum de la série filtrée (ensemble inférieure)
            vwmp_lower = group.loc[max_cumul_amount_index, 'price']                     #Déduction du prix médian bas pondéré par le volume
            return vwmp_lower  
        
        else : 
            return group['price'].mean()    #Cas particuliers (mesuré x3 occurences) soit (vwmp = price si échantillon = 1 ;  vwmp = 0 si échantillon = 0) -> utilisation de la fonction mean pour renvoyer ce résultat

    #-- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - -- - #symétrie

    if vwmp_type == 'upper':    #UPPER
    
        upper_cumulative_volume = series_sorted[series_sorted['cumul_amount'] <= total_volume_median]   #Filtrage de la série pour conserver la partie inférieure (>=) du volume cumulé 
    
        if not upper_cumulative_volume.empty:   #Si la taille de l'échantillon est >> 2
       
            min_cumul_amount_index = upper_cumulative_volume['cumul_amount'].idxmin()   #Identification de la ligne correspondante au volume cumulé maximum de la série filtrée (ensemble inférieure)
            vwmp_upper = group.loc[min_cumul_amount_index, 'price']                     #Déduction du prix médian bas pondéré par le volume
            return vwmp_upper  
    
        else : 
            return group['price'].mean()    #Cas particuliers (mesuré x3 occurences) soit (vwmp = price si échantillon = 1 ;  vwmp = 0 si échantillon = 0) -> utilisation de la fonction mean pour renvoyer ce résultat
    #----------
    else:
        raise ValueError("Mode non valide. Veuillez spécifier 'lower' ou 'upper'.")

##__Boucle_2__## 

def cleaning(df):   #Supprime les lignes vides (pas de transactions dans un intervalle de temps)
    
    df_cleaned = df.copy()                                                      #Copie pour ne pas modifier le df original
    df_cleaned = df_cleaned[~df_cleaned.apply(                                  #Filtrage des lignes où toutes les valeurs sont nulles
        lambda row: all(val == 0.0 or pd.isnull(val) for val in row), axis=1    
    )] 
    
    return df_cleaned

#_VISUALISE_#
       
def visualisation(cleaned_dict):    #Boucle de création de graphique
    
    # WIDGETS # 
    grid = widgets.GridspecLayout(1, 10) 
    grid[0, 0] = export_button
    grid[0, 1] = dropdown_ladder
    grid[0, 2] = dropdown_vwmp_type
    display(grid)  
       
    for key, df in reversed(list(cleaned_dict.items())):        #Boucle de création de graphique pour chaque dataframe
    
        exchange_name_splitted = (str(key)).split("df_")        #Extraction du nom de l'exchange
        exchange_name = exchange_name_splitted[1]

        candlestick = go.Figure(data=[go.Candlestick(x=df.index, #Création d'un graphique en chandelier japonais 
                                                      open=df['price_open'],
                                                      high=df['price_high'],
                                                      low=df['price_low'],
                                                      close=df['price_close']
                                                     )])

        candlestick.update_layout(title=f'Exchange - [{exchange_name}]', #Mise à jour du graphique
                                  yaxis_title='Price',
                                  height=300,  
                                  width=800,   
                                  margin=dict(l=70, r=50, t=50, b=20)
                                  )
       
        candlestick.add_trace(go.Scatter(x=df.index, y=df[f'{exchange_name}_vwap'], mode='lines', name='VWAP', line_color='blue'))      #Ajout courbe du Volume Weighted Average Price [VWAP]
        
        column_index = 9                                                                                                                #Index de la colonne 'vwmp_XXXer'(10ème colonne)
        column_name = df.columns[column_index]                                                                                          #Nom de la colonne 'vwmp_upper' ou 'vwmp_lower' 
        candlestick.add_trace(go.Scatter(x=df.index, y=df.iloc[:, column_index], mode='lines', name=column_name, line_color='grey'))    #Ajout courbe du Volume Weighted Median Price [VWMP] avec nom de série associé
        
        candlestick.show()      #Affichage du graphique crée pour le dataframe
    
    
#_EXPORT_CSV_#

def export_csv(export_folder):      #Fonction en cas de clic sur le wigdget 'Exporter en csv'
    
    concatenated_dict = {}                                  #Dictionnaire concaténé = cleaned_dict + 'synthese'     
    concatenated_dict = concatenate_output(cleaned_dict)    #Remplissage de concatenated_dict avec la fonction "concatenate_output"

    for key, df in concatenated_dict.items():   

        filename = (f"{key}.csv")                   #ID du fichier csv = nom du dataframe
        file_path = export_folder + filename        #Chemin de l'enregistrement
        df.to_csv(file_path, index=True)            #enregistre le dataframe en fichier .csv
    
    print(f"Export des fichiers csv avec succès (voir arborescence ci-dessous): {export_folder}")

    return concatenated_dict 

def concatenate_output(cleaned_dict):       #Assemblage des résultats dans un dataframe 'synthese' 
    
    df_output = None    #Déclaration d'un dataframe vide
    
    for key, df in cleaned_dict.items():        
        
        exchange_name_splitted = (str(key)).split("df_")   #Extraction du nom de l'exchange dans le nom du dataframe
        exchange_name = exchange_name_splitted[1]
                        
        if f'{exchange_name}_vwap' in df.columns:          #Check si la colonne 'vwap' existe dans le dataframe 
            
            vwap_column = df[f'{exchange_name}_vwap']      #Identifie la colonne 'wvap' de l'exchange
                        
            if df_output is None:                          #1ère itération
               df_output = vwap_column.to_frame()          #Copie de la colonne 'vwap' de l'exchange et intégration au dataframe de synthese
            else:
               df_output = pd.concat([df_output, vwap_column], axis=1)  #itération >= 2 : concaténe la colonne 'wvap' de l'exchange au dataframe de synthese
    
        concatenated_dict = cleaned_dict.copy()            #Duplication du dictionnaire d'entrée
        concatenated_dict['synthese'] = df_output          #Intégration du dataframe 'synthese' dans le nouveau dictionnaire
        
    return concatenated_dict

def on_export_button_click(b):      #Widget pour export csv
    
    notebook_dir = os.getcwd()                              #Chemin du notebook
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")      #Timestamp du lancement export
    export_folder = (notebook_dir) + '/../output/Export_' + (now) + '_freq-'+ (dropdown_ladder.value) + '_vwmp-' + (dropdown_vwmp_type.value) +'/'      #Création d'un dossier 'export' horodaté avec les paramètres sélectionnées à l'export
    
    os.mkdir(export_folder)         #Création du dossier 'Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X'
    export_csv(export_folder)       #Appel fonction 'export csv' 

export_button = widgets.Button(description="Exporter en CSV")

#_INITIALISATION_#

cleaned_dict = {}    #Dictionnaire des données agrégées et nettoyées

dropdown_ladder = select_time_ladder()              #init selection 'frequency' par défault
dropdown_vwmp_type = select_vwmp_type()             #init selection 'vwmp_type' par défaut
export_button.on_click(on_export_button_click)      #init button_click pour exporter en csv

aggregation(dropdown_ladder.value, dropdown_vwmp_type.value)    #initialisation par défaut

#### RESULTATS

Dans le cas où les widgets ne fonctionnerait pas dans le jupyter notebook :

1) Lancer l'aggrégation avec les paramètres souhaités et visualiser les résultats :

In [ ]:
aggregation('60min', 'lower')   

    #agg1 : ('60min', '30min', '5min')
    #agg2 : ('lower', 'upper')

2) Exporter les données avec les paramètres sélectionnés

In [ ]:
b =True
on_export_button_click(b)

Arborescence du projet : 

In [ ]:
##  .
##  ├── data
##  │   ├── bfly.csv
##  │   ├── bfnx.csv
##  │   ├── bnus.csv
##  │   ├── btrx.csv
##  │   ├── cbse.csv
##  │   ├── gmni.csv
##  │   ├── itbi.csv
##  │   ├── krkn.csv
##  │   ├── lmax.csv
##  │   ├── okcn.csv
##  │   └── stmp.csv
##  ├── livrable
##  │   └── livrable.ipynb
##  ├── output
##  │   ├── Export_YYYY-MM-DD_HH-MM-SS_freq-Xmin_vwmp-X
##  │   │   ├── df_all.csv
##  │   │   ├── df_bfly.csv
##  │   │   ├── df_bfnx.csv
##  │   │   ├── df_bnus.csv
##  │   │   ├── df_btrx.csv
##  │   │   ├── df_cbse.csv
##  │   │   ├── df_gmni.csv
##  │   │   ├── df_itbi.csv
##  │   │   ├── df_krkn.csv
##  │   │   ├── df_lmax.csv
##  │   │   ├── df_okcn.csv
##  │   │   ├── df_stmp.csv
##  │   │   └── synthese.csv
##  └── readme.txt